In [0]:
dbutils.fs.unmount("/mnt/data")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-16523530741427>, line 1
----> 1 dbutils.fs.unmount("/mnt/data")

File /databricks/python_shell/dbruntime/dbutils.py:158, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    156 exc.__context__ = None
    157 exc.__cause__ = None
--> 158 raise exc

ExecutionError: An error occurred while calling o408.unmount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory not mounted: /mnt/data; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory not mounted: /mnt/data
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:148)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:70)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.unmount(DBUtilsCore.scala:14

# Mounting the file from storage account to databricks

In [0]:
storage_account_name = "storagebyevin"
container_name = "inputs"
mount_point = "/mnt/data"

# Fetch the secret key
storage_account_key = dbutils.secrets.get(scope="databricksscope", key="storage")

# Mount the storage account
dbutils.fs.mount(
  source = f"wasbs://inputs@storagebyevin.blob.core.windows.net",
  mount_point = mount_point,
  extra_configs = {f"fs.azure.account.key.storagebyevin.blob.core.windows.net": storage_account_key}
)


True

In [0]:
%fs
ls /mnt/data/landing/

path,name,size,modificationTime
dbfs:/mnt/data/landing/orders.csv,orders.csv,224,1726787279000


## Reading the mounted file

In [0]:
df=spark.read \
    .format("csv") \
        .option("header", "true")  \
            .option("inferSchema", "true")  \
                .load("/mnt/data/landing/orders.csv")

In [0]:
df.show()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
| 1111111|2013-07-25 00:00:00|      11599|         CLOSED|
| 2222222|2013-07-25 00:00:00|        256|PENDING_PAYMENT|
| 3333333|2013-07-25 00:00:00|      12111|       COMPLETE|
| 4444444|2013-07-25 00:00:00|       8827|         CLOSED|
+--------+-------------------+-----------+---------------+



## Validation check-1 (Check if order_id is unique)

In [0]:
check=df.count()
count_of_order_id=df.select("order_id").distinct().count()
if check!=count_of_order_id:
    dbutils.fs.mv("/mnt/data/landing/orders.csv","/mnt/data/discarded")
    dbutils.notebook.exit("Duplicate order_id found")
else:
    print("All Good")



All Good


## Connecting an azure sql database to azure databricks
### I have created a table valid_order_status that contains valid order_status values

In [0]:
# Set the connection properties
jdbcHostname = "serverbyevin.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "database"
jdbcUsername = "evin"
jdbcPassword = "Sql-password"

jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase}"

# Connection properties
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Query to execute
query = "(SELECT * FROM valid_order_status) as temp"

# Load the data from the Azure SQL Database into a DataFrame
df2 = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)

# Show the DataFrame content
df2.show()


+---------------+
|    status_name|
+---------------+
|        ON_HOLD|
|PENDING_PAYMENT|
|     PROCESSING|
|         CLOSED|
| PAYMENT_REVIEW|
|SUSPECTED_FRAUD|
|       COMPLETE|
|        PENDING|
|      CANCELLED|
+---------------+



# Checking if order_status are valid

In [0]:
df.createOrReplaceTempView("check_status")
df2.createOrReplaceTempView("valid_status")

In [0]:
invalid = spark.sql("""
    SELECT c.order_status 
    FROM check_status c
    LEFT JOIN valid_status v ON c.order_status = v.status_name
    WHERE v.status_name IS NULL
""")

In [0]:
if invalid.count()>0:
    dbutils.fs.mv("/mnt/data/landing/orders.csv","/mnt/data/discarded")
    dbutils.notebook.exit("invalid order_status found")
else:
    dbutils.fs.mv("/mnt/data/landing/orders.csv","/mnt/data/staging")
